In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

class RAGBot:
    def __init__(self, api_key, model_name="meta-llama/llama-3.1-8b-instruct"):
        """
        Inisialisasi bot dengan koneksi ke OpenRouter (Provider Groq).
        """
        if not api_key:
            raise ValueError("API Key tidak boleh kosong!")

        # 1. Setup LLM (Otak)
        self.llm = ChatOpenAI(
            openai_api_key=api_key,
            openai_api_base="https://openrouter.ai/api/v1",
            model_name=model_name,
            temperature=0.0,
            model_kwargs={
                "extra_body": {
                    "provider": {
                        "order": ["Groq"], # Paksa pakai Groq biar ngebut
                        "allow_fallbacks": True
                    }
                }
            }
        )

        # 2. Setup Prompt (Instruksi)
        # Template ini otomatis menampung {context} dan {question}
        self.prompt = ChatPromptTemplate.from_template("""
        Kamu adalah asisten AI yang membantu menjawab pertanyaan berdasarkan data yang diberikan.
        
        INSTRUKSI:
        1. Gunakan HANYA informasi dari KONTEKS di bawah untuk menjawab.
        2. Jika jawaban tidak ada di konteks, katakan "Maaf, informasi tidak ditemukan dalam dokumen."
        3. Jawab dengan bahasa yang sopan dan jelas.

        KONTEKS DATA:
        {context}

        PERTANYAAN USER:
        {question}
        """)

        # 3. Setup Chain (Pipa Penghubung)
        # Prompt -> LLM -> Output Bersih
        self.chain = self.prompt | self.llm | StrOutputParser()

    def tanya(self, context: str, question: str) -> str:
        """
        Fungsi utama untuk bertanya.
        Input: 
            - context: Teks dokumen/data (String)
            - question: Pertanyaan user (String)
        Output: 
            - Jawaban AI (String)
        """
        try:
            print(f"🤖 Sedang berpikir (Model: {self.llm.model_name})...")
            
            # Eksekusi Chain
            jawaban = self.chain.invoke({
                "context": context,
                "question": question
            })
            
            return jawaban
        except Exception as e:
            return f"Terjadi Error: {e}"

# ==========================================
# CONTOH CARA PAKAI (MAIN PROGRAM)
# ==========================================
if __name__ == "__main__":
    # 1. Masukkan API Key OpenRouter kamu di sini
    MY_KEY = "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"  # <--- GANTI INI DENGAN KEY ASLI KAMU

    # 2. Buat Instance Bot
    bot = RAGBot(api_key=MY_KEY)

    # ---------------------------------------------------------
    # SKENARIO: Kamu punya data dari PDF/Database (Simulasi)
    # ---------------------------------------------------------
    data_dokumen = """
<table>
<thead>
<tr>
<th>Nomor</th>
<th>Item</th>
<th>Persentase Efisiensi</th>
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td>Alat Tulis Kantor</td>
<td>90,0%</td>
</tr>
<tr>
<td>2</td>
<td>Kegiatan Seremonial</td>
<td>56,9%</td>
</tr>
<tr>
<td>3</td>
<td>Rapat, Seminar dan sejenisnya</td>
<td>45,0%</td>
</tr>
<tr>
<td>4</td>
<td>Kajian dan Analisis</td>
<td>51,5%</td>
</tr>
<tr>
<td>5</td>
<td>Diklat dan Bimtek</td>
<td>29,0%</td>
</tr>
<tr>
<td>6</td>
<td>Honor Output Kegiatan dan Jasa Profesi</td>
<td>40,0%</td>
</tr>
<tr>
<td>7</td>
<td>Percetakan dan Souvenir</td>
<td>75,9%</td>
</tr>
<tr>
<td>8</td>
<td>Sewa Gedung, Kendaraan, Peralatan</td>
<td>73,3%</td>
</tr>
<tr>
<td>9</td>
<td>Lisensi Aplikasi</td>
<td>21,6%</td>
</tr>
<tr>
<td>10</td>
<td>Jasa Konsultan</td>
<td>45,7%</td>
</tr>
<tr>
<td>11</td>
<td>Bantuan Pemerintah</td>
<td>16,7%</td>
</tr>
<tr>
<td>12</td>
<td>Pemeliharaan dan Perawatan</td>
<td>10,2%</td>
</tr>
<tr>
<td>13</td>
<td>Perjalanan Dinas</td>
<td>53,9%</td>
</tr>
<tr>
<td>14</td>
<td>Peralatan dan Mesin</td>
<td>28,0%</td>
</tr>
<tr>
<td>15</td>
<td>Infrastruktur</td>
<td>34,3%</td>
</tr>
<tr>
<td>16</td>
<td>Belanja Lainnya</td>
<td>59,1%</td>
</tr>
</tbody>
</table>

    """

    # 3. User Bertanya
    pertanyaan_user = "Jelaskan isi table tentang apa"

    # 4. Panggil Fungsi 'tanya' (Masukkan Context & Question)
    hasil = bot.tanya(context=data_dokumen, question=pertanyaan_user)

    # 5. Tampilkan Hasil
    print("\n=== HASIL JAWABAN ===")
    print(hasil)

c:\conda\miniconda3\envs\torch\lib\site-packages\IPython\core\interactiveshell.py:3550: UserWarning: Parameters {'extra_body'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  exec(code_obj, self.user_global_ns, self.user_ns)


🤖 Sedang berpikir (Model: meta-llama/llama-3.1-8b-instruct)...

=== HASIL JAWABAN ===
Berdasarkan konteks data yang diberikan, saya dapat menyimpulkan bahwa tabel tersebut menampilkan informasi tentang efisiensi penggunaan anggaran atau biaya untuk berbagai kegiatan dan item.

Tabel tersebut mencakup 16 item, masing-masing dengan persentase efisiensi yang berbeda-beda. Persentase efisiensi ini mungkin menunjukkan seberapa efektif penggunaan anggaran atau biaya untuk masing-masing item.

Dengan demikian, tabel tersebut dapat digunakan untuk menganalisis dan memantau efisiensi penggunaan anggaran atau biaya dalam suatu organisasi atau proyek.
